# Average Articulation 
computed an average articulation of a disyllabic vowel utterances
- Created: 02 April 2020

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import librosa
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import librosa.display
from matplotlib import cm
from functools import partial

import sklearn
from sklearn.decomposition import PCA
from os import makedirs
import os
from os.path import join
import shutil
from IPython import display

%matplotlib inline

In [2]:
np_load_old = partial(np.load)
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
# np.random.seed(42)

In [3]:
EXP = '27_3'
# data_path = '../data/d_eval'
data_path = '../data/d_records/record_all'

In [4]:
# path = '../experiment_2/result/eval_{}/testset.npz'.format(EXP)
path = '../experiment/result/predict_{}/testset.npz'.format(EXP)
data = np.load(path)['param_sets']
data = data.reshape((data.shape[0]*data.shape[1],24))
df = pd.DataFrame(data)

In [5]:
with open(join(data_path,'syllable_name.txt')) as f:
    syllable_name = np.array([word.strip() for line in f for word in line.split(',')])
    syllable_name = np.array([ '%s;%s'%(item,str(idx+1)) for pair in syllable_name for idx, item in enumerate(pair)])
    
df['phonetic'] = syllable_name
df['phonetic'], df['position'] = df['phonetic'].str.split(';', 1).str
df = df.groupby(['phonetic','position']).mean().reset_index()
df_1 = df[df['position']=='1'] 
df_2 = df[df['position']=='2'] 

In [6]:
df.head()

,phonetic,position,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
0,7,1,0.362240,-4.948086,0.0,-4.773786,0.413505,0.523274,0.529874,-0.049247,...,-0.229543,-6.615771,-2.486380,0.634440,0.339982,0.640100,0.131710,-0.05,-0.05,-0.05
1,7,2,0.423540,-4.941220,0.0,-4.746430,0.437106,0.487670,0.519323,-0.047913,...,-0.198556,-6.523970,-2.437876,0.655362,0.301791,0.582998,0.125315,-0.05,-0.05,-0.05
2,E,1,0.269774,-4.266772,0.0,-4.477761,0.027697,0.875300,0.820077,0.016660,...,0.147353,-6.503864,-2.150339,0.952865,0.208222,0.367875,-0.227606,-0.05,-0.05,-0.05
3,E,2,0.229271,-4.228761,0.0,-4.453353,0.009420,0.882928,0.820617,0.018237,...,0.126598,-6.509444,-2.163414,0.963699,0.227558,0.357762,-0.224958,-0.05,-0.05,-0.05
4,M,1,0.504928,-5.099138,0.0,-4.370216,0.592763,0.365741,0.368865,-0.053409,...,0.281512,-6.052506,-1.761972,1.023947,0.320576,0.306108,0.089920,-0.05,-0.05,-0.05


## Create Speaker File

In [9]:
def create_speaker_file(df, file_path):
                        
    speaker_header_file = '../experiment/lib/templates/speaker_template_head.txt'
    speaker_tail_file = '../experiment/lib/templates/speaker_template_tail.txt'

    # Read speaker template
    speaker_head = open(speaker_header_file,'r').read()
    speaker_tail = open(speaker_tail_file,'r').read()

    # load vocaltract param name
    param_name = np.load('../experiment/lib/templates/speaker_param.npz')['name']
    
    pho = set(df['phonetic'])
    
    with open(file_path, 'w') as f:
        f.write(speaker_head)

        for p in pho:
            df_temp = df[df['phonetic']==p].copy()
            f.write('<shape name="{}">\n'.format(p))
            for i in range(24):
                f.write('<param name="{}" value="{:.2f}"/>\n'.format(param_name[i],df_temp[i].values[0]))
            f.write('</shape>\n')

        f.write(speaker_tail)

In [10]:
create_speaker_file(df_1, 'speaker_27_pred_1.speaker')
create_speaker_file(df_2, 'speaker_27_pred_2.speaker')